In [1]:
!pip install ecdsa


   ---------------------------------------- 0.0/149.3 kB ? eta -:--:--
   -------- ------------------------------- 30.7/149.3 kB 1.4 MB/s eta 0:00:01
   ---------------- ---------------------- 61.4/149.3 kB 825.8 kB/s eta 0:00:01
   -------------------------------- ------- 122.9/149.3 kB 1.0 MB/s eta 0:00:01
   -------------------------------------- 149.3/149.3 kB 813.0 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import secrets
from ecdsa import SECP256k1, ellipticcurve, numbertheory
from hashlib import sha256
curve = SECP256k1
G = curve.generator
order = curve.order
def sha256_hash(data):
    return int.from_bytes(sha256(data).digest(), 'big')
private_key_party1 = secrets.randbelow(order)
private_key_party2 = secrets.randbelow(order)

public_key_party1 = private_key_party1 * G
public_key_party2 = private_key_party2 * G
combined_public_key = public_key_party1 + public_key_party2

nonce_party1 = secrets.randbelow(order)
nonce_party2 = secrets.randbelow(order)

R_party1 = nonce_party1 * G
R_party2 = nonce_party2 * G

R_combined = R_party1 + R_party2

message = b"Secure transaction to the crypto vault"
e = sha256_hash(R_combined.to_bytes() + combined_public_key.to_bytes() + message) % order

partial_signature_party1 = (nonce_party1 + e * private_key_party1) % order
partial_signature_party2 = (nonce_party2 + e * private_key_party2) % order

s_combined = (partial_signature_party1 + partial_signature_party2) % order
signature = (R_combined, s_combined)

e_verification = sha256_hash(R_combined.to_bytes() + combined_public_key.to_bytes() + message) % order

left_hand_side = s_combined * G
right_hand_side = R_combined + e_verification * combined_public_key

if left_hand_side == right_hand_side:
    print("Signature is valid!")
else:
    print("Signature is invalid!")


Signature is valid!
